# Chapter 09 -- Panda Time Series and Date Handling

# Topics Covered:

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2009%20--%20Panda%20Time%20Series%20and%20Date%20Handling.ipynb?flush_cache=true/#Creating-and-manipulating-a-fixed-frequency-of-dates-and-time-spans"> Creating and manipulating a fixed-frequency of dates and time spans </a>

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2009%20--%20Panda%20Time%20Series%20and%20Date%20Handling.ipynb?flush_cache=true/#Time-Series-Walk-Through">Time Series Walk-Through</a>

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2009%20--%20Panda%20Time%20Series%20and%20Date%20Handling.ipynb#Returning-Unique-Levels-of-Categories">Returning Unique Levels of Categories</a>

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2009%20--%20Panda%20Time%20Series%20and%20Date%20Handling.ipynb#Return-a-Row-using-a-Minimum-Value">Return a Row using a Minimum Value</a>

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2009%20--%20Panda%20Time%20Series%20and%20Date%20Handling.ipynb#Return-a-Row-using-a-Maximum-Value">
Return a Row using a Maximum Value</a>

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2009%20--%20Panda%20Time%20Series%20and%20Date%20Handling.ipynb?flush_cache=true/#Convert-Time-Series-from-one-Frequency-to-Another">Convert time series from one frequency to another</a>

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2009%20--%20Panda%20Time%20Series%20and%20Date%20Handling.ipynb#Resources">Resources</a>

Chapter 8, <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2008%20--%20Python%20Date%2C%20Time%2C%20and%20%20Timedelta%20Objects.ipynb"> Understanding Date Time and TimeDelta objects </a> provided a short introduction to Python's built-in datetime capabilities.  In this chapter we illustrate pandas time series and date handling.  



In [ ]:
from datetime import date, time, datetime, timedelta
import numpy as np
import pandas as pd
from pandas import Series, DataFrame, Index
from numpy.random import randn as rnd
from IPython.display import Image
import warnings
warnings.simplefilter("ignore")


## Creating and manipulating a fixed-frequency of dates and time spans

The pd.date_range() method generates a DateTime Index which is applied to a panda Series or DataFrame to provide datetime interval indexing.  We will see examples of its construction methods.  And later we will utilize indexers taking advange of the Date TimeIndex.  

In [ ]:
rng = pd.date_range('1/1/2016', periods=30, freq='D')

In [ ]:
rng

Print the first 10 dates in the DateTimeIndex

In [ ]:
rng[:10]

Assemble a Series by using strings and integers for columns.  Map the year, month, and day value into a date timestamp using the pd.to_datetime() method.  Details for the pd.datetime() method are found <a href="http://pandas.pydata.org/pandas-docs/version/0.18.1/generated/pandas.to_datetime.html"> here</a>.

In [ ]:
df = pd.DataFrame({'year': ['2014', '2015', '2016'],
                   'month': [1, 2, 3],
                   'day': [1,2,3,]})
df1 = pd.to_datetime(df)
df1

Construct the Series 'b_rng' containing only business days using the pd.bdate() method and supplying start and end date.  

In [ ]:
start = datetime(2016, 1, 1)
end = datetime(2016, 12, 31)
b_rng = pd.bdate_range(start,end)

The Series implicitly creates a DatetimeIndex object.  

In [ ]:
type(b_rng)

In [ ]:
b_rng

Create a Series containing the last business day of the month for 2016.

In [ ]:
rng = pd.date_range(start, end, freq='BM')
ts = pd.Series(np.random.randn(len(rng)), index=rng)
ts.index

Returns the first 5 last business day of the month for 2016.

In [ ]:
ts[:5].index

 Returns the last business day of every other month in 2016.

In [ ]:
ts[::2]

## Time Series Walk-Through

We can begin combining features covered in previous chapters to conduct a walk-through of a simple   time series analysis.  

#### The Data

The data is the FHFA House Price Index (HPI) which is a broad measure of the movement of single-family house prices. It is a weighted, repeat-sales index, meaning that it measures average price changes in repeat sales or refinancing on the same properties. This information is obtained by reviewing repeat mortgage transactions on single-family properties whose mortgages have been purchased or securitized by Fannie Mae or Freddie Mac.  

Details about the data and how it is organized can be found <a href="https://catalog.data.gov/dataset/fhfa-house-price-indexes-hpis"> here </a>. This time series begins January 1991 and end August 2016.  Both the seasonally adjusted index 'index_sa' and the non-seaonally adjusted index 'index_nsa' set the index value at 100 for January 1991.  

#### The Inquiry

The three salient questions to answer are:

    1. Have U.S. aggregate home prices recovered their value since the Great Recession of 2008-2010? 
    2. Where are the highest and lowest values for 'traditional', 'purchase-only' homes in the U.S. in 2016? 
    3. How do the highest and lowest home values market segments compare to the aggregate U.S. home prices?
    
#### The Approach

The input .csv file located <a href="some_URL"> here </a>.  

The file is composed of two parts.  Part 1, rows 2 to 3081 are records for the aggregate market groups at the Census Division level.  The frequency interval is monthly.  

This portion of the file has 10 columns containing values for major market segments and the U.S. aggregate prices.  The price indicies are both seasonally adjusted and non-seasonally adjusted values.  The end-result DataFrame will be called 'df_us'.

Part 2, rows 3082 to 96,243 are more granular with a quarterly frequency interval. The major U.S. market segments are broken into smaller geographics areas with just the non-seasonaly adjusted home index value. Accordingly, the 10th column 'index_sa' contains no values at this location to the end of the file. The end-result DataFrame will be called 'df_states'.


#### Part 1 of .csv File to Construct the 'df_us' DataFrame

Start with a test read of the entire .csv file.  The pd.read_csv method has the one required arguement, the input file name to create the DataFrame 'df_all'.

In [ ]:
df_all = pd.read_csv("data/HPI_master.csv")

Inspect the first 5 rows to determine if the read_csv() method is giving the expected results.

In [ ]:
df_all.head()

We need to combine the year and period fields into a <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2008%20--%20Date%2C%20Time%2C%20and%20%20Timedelta%20Objects.ipynb#String-Literal-Mapped-to-datetime-timestamp"> datetime timetamp </a>.  The .csv file in the cell above is read without any datetime parsing for the fields, 'yr' and 'period'.  We could post-process these fields to construct the appropriate date timestamp values.  

A better approach is below.  The parse_dates= argument allows a <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2002%20--%20Data%20Structures.ipynb#dictionary"> dictionary </a> object with the key being the arbitrary name of the new column created and the key values indicating which fields are to be read in the .csv file.  Recall that Python indexes have a start position of 0.  In the .csv file, these fields are the 7th and 8th column position.

Sometimes, you may need to create your own date-parser, analogous to building a user-defined SAS INFORMAT to map field values into a datetime object.  This is particularly true in cases where the date value is stored as component values in multiple fields.   

The nrows= argument value is set to 3080.  

In [ ]:
df_us = pd.read_csv("data/HPI_master.csv",
                 parse_dates={'date_idx': [6,7]},
                 nrows=3080)

In [ ]:
df_us.shape

Validate column names and their data types.  Confirm the date parser constructed the column 'date_idx' as a datetime object.

In [ ]:
df_us.info()

Check for missing values.

In [ ]:
df_us.isnull().any()

Set the 'date_idx' column as the <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2005%20--%20Understanding%20Indexes.ipynb#Setting-and-resetting-Indicies">index</a> on the DataFrame.

In [ ]:
df_us.set_index("date_idx", inplace=True, drop=False)

Indexing on the datetime column 'date' creates a 'datetime-aware' DateTimeIndex.  

In [ ]:
df_us.index

Get the earlies and lastest date values in the 'df_us' DataFrame.

In [ ]:
print('Earliest date is:', df_us.date_idx.min())
print('Latest date is:', df_us.date_idx.max())

We see from the list of columns returned from the .info() attribute above we have several categorical columns.  We need to understand their levels.  Earlier, we saw the .describe() method used for numerical columns.  In the example below specifying the 'include=' argument returns a description of string columns.  

Recall that dtype 'O' (not zero) indicates string values for a Series or DataFrame column.

In [ ]:
df_us.describe(include=['O'])

The 'place_name' column has 10 unique levels or values.  We can examine these values with the .unique() attribute.

In [ ]:
df_us.place_name.unique()

Continue by setting an <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2005%20--%20Understanding%20Indexes.ipynb#Setting-and-resetting-Indicies"> index </a> on the column 'place_name'.  Construct the 'df_us_plot' DataFrame with rows for 'place_name' equal to 'United States'.  The .loc indexer allows row slicing which is covered in detail <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2005%20--%20Understanding%20Indexes.ipynb#.loc-Indexer">here</a>.  

In [ ]:
df_us.set_index('place_name', inplace=True, drop=False)
df_us_plot = df_us.loc['United States']

#### Answer to question 1
    Have U.S. aggregate home prices recovered their value since the Great Recession of 2008-2010?

During the Great Recession of 2008-2010, home prices across the U.S. declined dramatically.  The aggregate U.S. home price index has recovered all of the losses since that time and have continued to experience steady growth. 

Create the DataFrame 'df_us_3' to select the rows with the values indicated below.

In [ ]:
df_us_3 = df_us.loc[['West South Central Division', 'United States', 'Pacific Division']]

In [ ]:
df_us_3.info()

#### Part 2 of .csv File to Construct the 'df_states' DataFrame

Use to 'skiprows=' argument to begin reading at row 3082.  We use a tuple to specify the columns labels. 

Beginning with row 3082 the values for the field 'index_sa' are missing.  Begin the read at row 3082 until end of file.  And since the default is to key off column names, supply column labels with a<a href= "http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2002%20--%20Data%20Structures.ipynb#tuple"> tuple</a> of names.  The usecols= argument uses a tuple of integers to indicate which fields are to be read from the .csv file.  Header=None is to prevent the reader from building column names at row position nrows-1, which in our case contains data values.

In [ ]:
df_states = pd.read_csv("data/HPI_master.csv",                 
            skiprows=3082,
            usecols=(0, 1, 2, 3, 4, 5, 6, 7, 8),
            names=('hpi_type', 'hpi_flavor', 'frequency', 'level', 'place_name', 'place_id', 'yr', 'period', 'index_nsa'),
            header=None)

The columns 'yr' and 'period' are read as string values and need to be converted to datetime values.  The operation below creates the new column 'date_str' by:

    1. Concatenating 'yr' with 'Q' with 'period' to form a date string 
    2. The date string is passed to the pd.to_datetime function creating the dateime column 'date_idx'

    1. String concatenation operation to form YYYYq.

In [ ]:
df_states["date_str"] = df_states['yr'].map(str) + 'Q' + df_states['period'].map(str)

Display the 'date_str' value using the <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2005%20--%20Understanding%20Indexes.ipynb#.iloc-Indexer"> iloc indexer</a> which returns row and column location by integer positions.

In [ ]:
df_states.iloc[0,-1]

    2. Convert the 'date_str' column into a panda datetime column called 'date_idx'. 

In [ ]:
df_states['date_idx'] = pd.to_datetime(df_states['date_str'])

Inspect the first 5 records in the 'df_states' DataFrame.

In [ ]:
df_states.head()

Return the number of rows and columns in the DataFrame.

In [ ]:
df_states.shape

The following SAS Data Step reads the same .csv file using FIRSTOBS= to begin reading from the arbitary row position 3082.

````
    /********************************/
    /* c09_read()_csv_df_states.sas */
    /********************************/
    data df_states;
          infile 'data/HPI_master.csv' delimiter=',' missover dsd firstobs=3082; 
          informat hpi_type $12.
                   hpi_flavor $16.
                   frequency $9.
                   level $28.
                   place_name $33.
                   place_id $8.
                   yr $5.
                   period $6.
                   index_nsa 8.;
             input hpi_type $
                   hpi_flavor $
                   frequency $
                   level $
                   place_name $
                   place_id $
                   yr $
                   period $
                   index_nsa ;
````

Display the first 5 rows of the SAS data set df_states.  In the SAS code example below the 'yr' and 'period' variables are combined to create the SAS datetime variable 'date_idx'.

In [ ]:
Image(filename='output/df_states_output.JPG')  

Inspect values for the categorical columns.

In [ ]:
df_states.describe(include=['O'])

## Returning Unique Levels of Categories

Get unique values for column 'level'

In [ ]:
df_states.level.unique()

PROC SQL used to obtain unique values from the variable 'level'.

````
    /******************************************************/
    /* c09_select_unique_level.sas                        */
    /******************************************************/
    56       proc sql;
    57          select unique level
    58          from df_states;
    59       quit;
````

In [ ]:
Image(filename='output/level_unique_values.JPG')  

Start by creating a <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2002%20--%20Data%20Structures.ipynb#list">list</a> called 'lvls' using the .select_dtypes() attribute to include columns with dtype value 'O' selecting columns with string values.

In [ ]:
lvls = list(df_states.select_dtypes(include=['O']).columns)

Inspect the 'lvls' list.

In [ ]:
lvls 

If we had a long list of columns needing to determine unique values the approach above becomes tedious; an iterative approach is called for using the for statement shown below.

In [ ]:
for item in lvls[:4]:
   print(item, 'levels are:', df_states[item].unique())    

Using the levels for the categorial columns displayed above, filter the 'df_state' DataFrame to match the 'df_us' Dataframe.  For this row slicing operation, create a boolean mask and combine it with the .loc indexer described <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2005%20--%20Understanding%20Indexes.ipynb#Mixing-.loc-Indexer-with-Boolean-Operators">here</a>.  This is a common pattern for filtering rows or column values.  

Create a boolean mask for the filtering criteria.

In [ ]:
mask = (df_states['hpi_type'] == 'traditional') & (df_states['hpi_flavor'] == 'purchase-only') & \
       (df_states['level'] == 'State')

Apply the mask.  Notice the 'df_states' DataFrame is updated in place with this assignment.  The original 'df_states' DataFrame had 96244 rows. 

In [ ]:
df_states = df_states.loc[mask]

The .shape attribute returns the new row and column count for the 'df_states' DataFrame.

In [ ]:
df_states.shape

#### Minimums and Maximums for Part 2, 'df_states' DataFrame

Index the column 'index_nsa' in order to find the maximum and minimum for the 'df_states' DataFrame.

In [ ]:
df_states.set_index('index_nsa', inplace=True, drop=False)

In [ ]:
print('Max value for index_nsa:', df_states['index_nsa'].max())
print('Min value for index_nsa:', df_states['index_nsa'].min())

PROC SQL for finding min and max for the variable 'index_nsa'.

````
    /******************************************************/
    /* c09_min_max_index_nsa.sas                          */
    /******************************************************/
37      proc sql;
38         select max (index_nsa) as max_index_nsa,
39                min (index_nsa) as min_index_nsa
40         from df_states
41         where hpi_type ='traditional' and  hpi_flavor = 'purchase-only' and
42          level = 'State';
43      quit;
````

In [ ]:
Image(filename='output/max_min_index_nsa.JPG')

#### Filter for  2016

For 2016, return the rows with the largest and smallest value for 'index_nsa'.  

In order to find the row with the lowest 'index_nsa' value, the example below uses a multi-step process:
    1. Create a boolean mask filtering the rows with 'date_idx' between January 1, 2016 and December 31, 2016
    2. Use the .loc indexer applying the mask to create the 'df_2016' DataFrame
    3. Use the .idxmin() attribute to return the row having the minimum 'index_nsa' value
    4. Use the .idxmax() attribute to return the row having the maximum 'index_nsa' value
    
Keep in mind that the 'df_states' DataFrame has already been filtered previously.

Create the boolean mask for the filtering criteria.

In [ ]:
mask1 = (df_states['date_idx'] >= '2016-01-01') & (df_states['date_idx'] <= '2016-12-31')

Apply the mask using the <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2005%20--%20Understanding%20Indexes.ipynb#.loc-Indexer">.loc indexer</a> 

In [ ]:
df_2016 = df_states.loc[mask1]

## Return a Row using a Minimum Value

Use the .idxmin() method to return the minimum 'index_nsa' value.

In [ ]:
df_2016.ix[df_2016['index_nsa'].idxmin()]

## Return a Row using a Maximum Value

Use the .idxmax() attribute to return the row with the maximum 'index_nsa' value.

In [ ]:
df_2016.ix[df_2016['index_nsa'].idxmax()]

The .min and .max attribute return minimum and maximum respectively.  The .idxmin() attribute returns the entire row as a Series.  This enables the ability to use other column values, like 'place_name' or 'place_id' as further filtering and selection criteria.

The .min() and .max attributes return a scalar.

In [ ]:
print("2016 minimum value for 'index_nsa':", df_2016.index_nsa.min())
print("2016 maximum value for 'index_nsa':", df_2016.index_nsa.max())

The SAS Data Step above used to the read .csv file does not create a SAS datetime variable.  This is illustrated below. 

The SAS example below uses a pair of PUT functions nested inside the YYQ function to create the SAS date variable 'date_idx'.  The PUT functions map the variables 'yr' and 'period' from character to numeric.  The YYQ function described <a href="http://support.sas.com/documentation/cdl/en/lrdict/64316/HTML/default/viewer.htm#a000199346.htm">here</a> returns a SAS datetime values from year and quarter values.

The Data Step below is continued from the SAS Data Step example above used to read the .csv file.

````
    /******************************************************/
    /* c09_min_max_index_nsa_for2016.sas                  */
    /******************************************************/
    12      date_idx=yyq(put(yr,8.),put(period,8.));
    13      format date_idx yyq10.;
    14      
    15      proc sql;
    16         select max (index_nsa) as max_index_nsa,
    17                min (index_nsa) as min_index_nsa
    18         from df_states
    19         where hpi_type ='traditional' and
    20               hpi_flavor = 'purchase-only' and
    21               level = 'State' and
    22               date_idx between '01Jan2016'd and '31Dec2016'd;
    23      quit;
````

In [ ]:
Image(filename='output/2016_min_max.JPG')  

#### Answer to question #2

    2. Where are the highest and lowest values for 'traditional', 'purchase-only' homes in the U.S. in 2016? 
    
**Lowest home prices are in the state of Connecticut** and the **highest are in Washington, D.C.**  

Recall the 'df_states' DataFrame was filtered for column 'hpi_type' = 'traditional' **and** column 'hpi_flavor' = 'purchase-only' **and** level='state'.  This DataFrame was subsequently filtered to create the DataFrame 'df_2016' in which we searched for the lowest and highest home index values.

## Convert Time Series from one Frequency to Another

In order to answer the last question:
    How do the highest and lowest home values market segments compare to the aggregate U.S. home prices?
    
Information from the 'df_states' and 'df_us' DataFrame need to be combined.  The 'df_us' monthly values need to be downsampled to quarterly.  In other words, aggregated to a lower frequency.  In doing so, the frequency in both DataFrames become quarterly. 

In order to answer the 3rd question above, we need to conduct the following:
    1. Set the index for 'df_us' DataFrame to the column 'date_idx'
    2. Resample the 'df_us' DataFrame aggregating from monthly to quarterly using mean values
    3. Create a column in the 'df_us' labeled 'place_name' with the value "U.S. Aggregate".  
    4. Extract rows from 'df_states' DataFrame for lowest and highest home values using the column 'place_name' 
    5. Merge (concatenate) the resampled 'df_us' DataFrame with the high & low value rows from the extract operation
    6. Plot the resulting 'DataFrame'


Start by examining the quarterly date values from the 'df_states' DataFrame... 

In [ ]:
 df_states.iloc[0:4, -1]

...and compare with the monthly date values from the 'df_us' DataFrame.

In [ ]:
df_us.iloc[0:4, 7]

#### 1. Set the index for 'df_us' DataFrame to the column 'date_idx'

In [ ]:
df_us.set_index('date_idx', inplace=True, drop=False)

####    2. Resample the 'df_us' DataFrame aggregating from monthly to quarterly using mean values

pandas can perform resampling operations during frequency conversion (in this case, an aggregation of monthly values into quarterly values for the 'df_us' DataFrame). This is a common pattern in time series analysis.

The .resample() method is a time-based <a href=" ">needs a link to groupby</a> operation, followed by a reduction method on each of its groups.  Frequency conversion doc is found <a href="http://pandas.pydata.org/pandas-docs/stable/timeseries.html#frequency-conversion"> here</a>.  The .resample() method accepts frequency offset suffixes that are listed <a href="http://pandas.pydata.org/pandas-docs/stable/timeseries.html#anchored-offsets"> here</a>. 
 
In order to align the monthly date values found in the 'df_us' DataFrame, with the quarterly date values in the 'df_states' DataFrame, use the 'QS' date offset.  'QS' sets the date value frequency to quarterly with year ending in December.  The resulting DataFrame is called 'df_us_qtr'.

In [ ]:
df_us_qtr = df_us.resample('QS').mean()

#### 3. Create a column in the 'df_us' labeled 'place_name' with the value "U.S. Aggregate". 

In [ ]:
df_us_qtr['place_name'] = 'U.S. Aggregate'

#### 4. Extract rows for the lowest and highest home values using the df_us_qtr['place_name'] column

 Set the <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2005%20--%20Understanding%20Indexes.ipynb#Setting-and-resetting-Indicies">index</a> to the column 'place_name' and extract the rows using the <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2005%20--%20Understanding%20Indexes.ipynb#.loc-Indexer">.loc indexer</a>.

In [ ]:
df_states.set_index('place_name', inplace=True, drop=False)

Create a boolean mask using the logical OR operator ( | ).

In [ ]:
mask = (df_states['place_name'] == 'Connecticut') | (df_states['place_name'] == 'District of Columbia')

Apply the mask using the <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2005%20--%20Understanding%20Indexes.ipynb#.loc-Indexer">.loc indexer</a> to create the 'hi_lo' DataFrame.

In [ ]:
hi_lo = df_states.loc[mask]

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2005%20--%20Understanding%20Indexes.ipynb#Setting-and-resetting-Indicies">Reset the index</a> since the 'hi_lo' DataFrame will be concatenated (merged) with the resampled 'us_df' DataFrame.

In [ ]:
hi_lo.reset_index(drop=True, inplace=True)

#### 5. Merge (concatenate) the resampled 'df_us' DataFrame with the high & low value rows from the extract operation

Reset the index, since the 'df_us_qtr' DataFrame will be merged with the 'hi_lo' DataFrame created above.  

In [ ]:
df_us_qtr.reset_index(inplace=True)

Create a <a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/blob/master/Chapter%2002%20--%20Data%20Structures.ipynb#list">list</a> of DataFrames to be included in the concatenation operation.

In [ ]:
frames = [df_us_qtr, hi_lo]

#### Answer to Question 3.
    3. How do the highest and lowest home values market segments compare to the aggregate U.S. home prices?

In the plot above, we see that the aggregate U.S. home price index regained its losses from the Great Recession (2008-2010) beginning around 2015.  Since then it has seen steady growth and has exceeded the peak from the pre-recession values.

In contrast, the home value index for Connecticut is just now recovering its value lost during the Great Recession.  However, it has not recovered its pre-recession value.

Since 2011, the aggregate U.S. home price index has been growing faster than that of Connecticut, but not even close to the growth rate for Washington, D.C.

## Resources 

pandas Time Series and Date functionality doc located <a href="http://pandas.pydata.org/pandas-docs/stable/timeseries.html">here</a>.

pandas datetime Indexing doc located <a href="http://pandas.pydata.org/pandas-docs/stable/timeseries.html#datetime-indexing">here</a>.

pandas cookbook for <a href="http://pandas.pydata.org/pandas-docs/stable/cookbook.html#timeseries">timeseries</a>.

Chapter 10, Time Series, "Python for Data Analysis, by Wes McKinney, located <a href="http://shop.oreilly.com/product/0636920023784.do">here</a>.

SAS 9.4 Language Reference: Concepts, 5th ed., <a href="http://support.sas.com/documentation/cdl/en/lrcon/68089/HTML/default/viewer.htm#n0q9ylcaccjgjrn19hvqnd9cte8p.htm"> Dates, Times, and Intervals

## Navigation

<a href="http://nbviewer.jupyter.org/github/RandyBetancourt/PythonForSASUsers/tree/master/"> Return to Chapter List </a>    